## Способы нахождения упоминаний товаров в продуктах.

1. Самый "топорный" и простой - составить словарь из данных наазваний продуктов. Кажется, что он покрывает достаточно много, однако, должен еще и вытаскивать ненужные вещи типа прилагательных и коллокаций из раздела "первый на рынке".
1.1 Здесь еще можно попробовать собрать словарь из самих отзывов с помощью bootstrapping - если мы увидим, что ревью похожи друг на друга по формулам.
2. Rule-based подход - парсер типа natasha неплохо подходит для задачи, если мы знаем, что названия продуктов "похожи" на именованные сущности. Так как наш датасет тематичен, можно предполоожить, что это будет хорошо.
3. Machine learning-based подход - например, применение CRF к датасету с pos-тэгами и другими характеристикмаи (например, капитализация). Звучит здорово, но требует предварительной ручной работы.

In [16]:
import pandas as pd

df = pd.read_json('/Users/kseniadudiy/Downloads/Magazine_Subscriptions.json', lines=True)

In [17]:
reviews = df['reviewText']
reviews

0        for computer enthusiast, MaxPC is a welcome si...
1        Thank god this is not a Ziff Davis publication...
2        Antiques Magazine is a publication made for an...
3        This beautiful magazine is in itself a work of...
4                                A great read every issue.
                               ...                        
89684    This was a nice surprise for my boyfriend. He ...
89685    Magazine looks like it is printed on recycled ...
89686    cant go wrong with an SI subscription\nvery pl...
89687    This magazine is by far my all time favorite o...
89688               Nice magazine. Good info and articles.
Name: reviewText, Length: 89689, dtype: object

Пробуем собрать словарь по шаблонаам с помощью bootstrapping.

Понимаем, что у нас большая часть именованных сущностей связана с журналами и их публикацией. Препроцессим текст так, чтобы на выходе мы получили списки со встроенными словарями, потом итерируемся по ним по шаблону.

Так как наша область крайне мала, мы знаем, что в наш словарь мы можем вручную добавить максимально употребимые слова: magazine, journal, publication.

X - наша именованная сущность (т.е. название журнала). У X должны быть следующие характеристики (некоторые из которых обязательны):

1. Заглавная буква в начале (понятно, что некоторые, как я, например, любят писать вообще без заглавных букв, но мы это опустим) - обязательно.
2. Слово "magazine" после.
3. Грамматические характеристики: noun/adj.
4. Глагол после.
5. Слова в начале предложения не анализируются - опять же, сложно ожидать от пишущих отзывы максимальной грамматичности, однако, сама именованная сущность по правилам не может находиться в первой позиции, ее обязательно предваряет артикль - это правило кажется максимально общеупотребимым, поэтому мы позволим себе на него опереться.

In [57]:
from nltk import pos_tag
def get_lists(x):
    l1 = x.split(' ')
    l2 = []
    if len(l1) > 1:
        a = pos_tag(l1)
        for item in a:
            dic = {}
            dic['word'] = item[0]
            if item[0][0].isupper():
                dic['letter'] = 'U'
            else:
                dic['letter'] = 'L'
            if len(item) > 1:
                dic['pos'] = item[1]
            else:
                dic['pos'] = 'None'
            l2.append(dic)
    return(l2)

In [58]:
print(get_lists('An rat ate my f')) 

[{'word': 'An', 'letter': 'U', 'pos': 'DT'}, {'word': 'rat', 'letter': 'L', 'pos': 'NN'}, {'word': 'ate', 'letter': 'L', 'pos': 'NN'}, {'word': 'my', 'letter': 'L', 'pos': 'PRP$'}, {'word': 'f', 'letter': 'L', 'pos': 'NN'}]


In [59]:
#чистим текст и сразу превращаем в нужные нам списки
import re
f = []
for el in reviews:
    d = re.findall("[\w']+|[.,!?;]", str(el))
    for item in d:
        if re.match('[.,?!;:-]', item) != None:
            d.remove(item)
        else:
            pass
    new = ' '.join(d)
    f.append(new)

In [60]:
fin = []
for item in f:
    fin.append(get_lists(item))
print(fin[:10])

[[{'word': 'for', 'letter': 'L', 'pos': 'IN'}, {'word': 'computer', 'letter': 'L', 'pos': 'NN'}, {'word': 'enthusiast', 'letter': 'L', 'pos': 'NN'}, {'word': 'MaxPC', 'letter': 'U', 'pos': 'NNP'}, {'word': 'is', 'letter': 'L', 'pos': 'VBZ'}, {'word': 'a', 'letter': 'L', 'pos': 'DT'}, {'word': 'welcome', 'letter': 'L', 'pos': 'JJ'}, {'word': 'sight', 'letter': 'L', 'pos': 'NN'}, {'word': 'in', 'letter': 'L', 'pos': 'IN'}, {'word': 'your', 'letter': 'L', 'pos': 'PRP$'}, {'word': 'mailbox', 'letter': 'L', 'pos': 'NN'}, {'word': 'i', 'letter': 'L', 'pos': 'NN'}, {'word': 'can', 'letter': 'L', 'pos': 'MD'}, {'word': 'remember', 'letter': 'L', 'pos': 'VB'}, {'word': 'for', 'letter': 'L', 'pos': 'IN'}, {'word': 'years', 'letter': 'L', 'pos': 'NNS'}, {'word': 'savorying', 'letter': 'L', 'pos': 'VBG'}, {'word': 'every', 'letter': 'L', 'pos': 'DT'}, {'word': 'page', 'letter': 'L', 'pos': 'NN'}, {'word': 'of', 'letter': 'L', 'pos': 'IN'}, {'word': 'boot', 'letter': 'L', 'pos': 'NN'}, {'word': 'as

In [68]:
from nltk.corpus import stopwords
words = ['magazine', 'journal', 'publication']
first = ['NN', 'NNP', 'NNS', 'JJ', 'JJR', 'JJS']
stops = list(stopwords.words('english'))
for item in fin:
    for element in item:
        i = item.index(element)
        if i > 0:
            if element['letter'] == 'U':
                if element['word'].lower() not in stops:
                    if element['pos'] in first:
                        if i < len(item)-1:
                            a = item[i+1]
                            if a['letter'] == 'U':
                                if element['word'].lower() not in stops:
                                    l = []
                                    l.append(element['word'])
                                    l.append(a['word'])
                                    words.append(' '.join(l))
                                else:
                                    words.append(element['word'])
                        else:
                            words.append(element['word'])
print(words)                

['magazine', 'journal', 'publication', "PC's Anyone", 'CD ROM', 'CD ROM', 'Ziff Davis', 'Driver Technical', 'Eating Utensil', 'Utensil Collection', 'Collection Spanish', 'Spanish Culture', 'Overall Antiques', 'Maximum PC', 'PC MPC', 'Period As', 'Maximum PC', 'Maximum PC', 'Maximum PC', 'Foreign Affairs', 'PC Always', 'Future US', 'US Customer', 'Customer Service', 'Maximum PC', 'Best PC', 'Maximun PC', 'PC The', 'PC World', 'World Maximum', 'Maximum PC', 'PC World', 'PC World', 'World Maximum', 'Maximum PC', 'PC World', 'Maximum PC', 'PC The', 'Windows This', 'PC Accelerator', 'Accelerator God', 'God Bless', 'PC Accelerator', 'Great PC', 'PC Accelerator', "PC'er", 'Maximum PC', 'New England', 'England Colonial', 'Colonial This', 'Ask Handyman', 'Handyman Here', 'Workshop Tips', 'Tips In', 'Handy Hints', 'Hints More', 'Great Goofs', 'Goofs Funny', 'New England', 'Products Section', 'Section These', 'Family Handyman', 'Todays Homeowner', 'Old House', 'Family Handyman', 'Handyman For', '

Отлично, что-то получилось! Давайте посчитаем повторяющиеся элементы - именно они и будут названиями.

In [73]:
from collections import Counter

cnt = Counter
imp = [key for key, _ in cnt.most_common(50)]

[('Kindle Fire', 1012),
 ('New Yorker', 847),
 ('National Geographic', 762),
 ("Reader's Digest", 416),
 ('Kindle I', 326),
 ('Amazon I', 323),
 ('TV Guide', 306),
 ('New York', 301),
 ('Popular Science', 270),
 ('Southern Living', 258),
 ('U S', 249),
 ('Food Network', 243),
 ('Magazine', 241),
 ('Amazon', 232),
 ('Good Housekeeping', 232),
 ('Vanity Fair', 230),
 ('Scientific American', 223),
 ('Real Simple', 219),
 ('Entertainment Weekly', 219),
 ('Thanks', 215),
 ('Marie Claire', 206),
 ('Popular Mechanics', 186),
 ('US Weekly', 173),
 ('Consumer Reports', 156),
 ("Men's Health", 151),
 ('Bon Appetit', 148),
 ('Country Living', 145),
 ('Martha Stewart', 145),
 ('Kindle', 142),
 ('Magazine I', 137),
 ('Conde Nast', 130),
 ('Readers Digest', 128),
 ('Better Homes', 121),
 ('Sports Illustrated', 118),
 ('Fire HD', 117),
 ('Fire I', 115),
 ('United States', 110),
 ('Family Handyman', 109),
 ('Golf Digest', 101),
 ('Cooking Light', 99),
 ('Old House', 95),
 ('Science Fiction', 95),
 ('N

Похоже на правду. Конечно, попали ненужные элементы, но их не так много.

## Собираю n-граммы

In [ ]:
#чистим корпус

alte = ' '.join(f)
wrds = alte.split(' ')
for item in wrds:
    i = wrds.index(item)
    if item in stops:
        wrds.pop(i)

print(wrds)

In [ ]:
neue = []
for item in wrds:
    neue.append(item.lower)

corpus_len = len(neue)
freq = cnt(neue)

clean_text = ' '.join(neue)

In [ ]:
colls = []
n = 0
for c in clean_text:
    for item in imp:
        found = re.findall(item, c)
        if found:
            n += 1
        for f in found:
            collocations.append((f[0], f[1]))
            collocations.append((f[1], f[2]))

In [ ]:
cnt(collocations).most_common(50)

## Метрики 